In [ ]:
# ===============================================================
# 03_kerr_flux_sim.ipynb  —  Kerr-enhanced quaternionic flux (shooting version)
# ===============================================================
!pip install --quiet scipy

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from math import sqrt, pi

# ---------- 常數與黑洞參數 ---------------------------------------
G, c = 6.67430e-11, 2.99792458e8
M_sun, M_pl = 1.98847e30, 2.176434e-8

M_bh   = 4.0e6 * M_sun
a_star = 0.5

r_g_m  = 2*G*M_bh/c**2
print(f"r_g ≈ {r_g_m/1e3:.2e} km,  a* = {a_star}")

# ---------- 無因次變數 ------------------------------------------
a_hat  = a_star
Mratio = M_bh / M_pl               # ≈ 3.65×10^44

Δ  = lambda x: x**2 - x + a_hat**2
dΔ = lambda x: 2*x - 1

κ = 2.0e-2
λ_tilde = lambda x: 2*κ*Mratio / x**3   # curvature–driven coupling

def rhs(x, y):
    ψ, χ = y
    src  = x**2 * λ_tilde(x) * ψ*(ψ**2 - 1)
    χp   = (src - (2*x*Δ(x)+x**2*dΔ(x))*χ) / (x**2 * Δ(x))
    return [χ, χp]

# ---------- 由外射擊：設定外層邊界條件 ---------------------------
x_outer = 1.0e3                      # r̂ = 1000
def integrate(A):
    ψ0, χ0 = A/x_outer, -A/x_outer**2
    sol    = solve_ivp(rhs, (x_outer, 2.0), [ψ0, χ0],
                       t_eval=np.geomspace(x_outer, 2.0, 1500),
                       method="Radau", rtol=1e-7, atol=1e-10,
                       max_step=5.0)
    return sol.t, 2*sol.y[0]         # 回傳 r̂ 與 ε

# ---------- 掃描 A 以命中 ε(10 r_g) ≈ 0.1 ------------------------
targets = [0.05, 0.1, 0.2]          # 想要的 ε(10 r_g)
solutions = []

for tgt in targets:
    # 粗調：在對數尺度上找 A
    As = np.logspace(-4, 1, 60)
    bestA, bestε = None, None
    for A in As:
        x, ε = integrate(A)
        ε10  = ε[np.argmin(np.abs(x-10))]
        if bestε is None or abs(ε10-tgt) < bestε:
            bestε, bestA = abs(ε10-tgt), A
    solutions.append((bestA, bestε))

bestA = min(solutions, key=lambda k:k[1])[0]
print(f"Chosen A = {bestA:.3e}")

x_arr, ε_arr = integrate(bestA)

# ---------- analytic ansatz -------------------------------------
r_s0_hat = (8.5e3 * 3.085677e16)/r_g_m
boost    = 1.0 / (1 - a_hat/x_arr)
ε_ansatz = 2*np.tanh(x_arr/(r_s0_hat*boost))

# ---------- 繪圖 -------------------------------------------------
plt.figure(figsize=(7,5))
plt.loglog(x_arr, ε_arr, lw=2, label="Numerical ε(r)")
plt.loglog(x_arr, ε_ansatz, '--', lw=2, label="Analytic ansatz")
plt.axvline(10, ls=':', color='k'); plt.text(10.4, 1e-2, "10 r_g")
plt.xlabel(r"$\hat r = r/r_g$"); plt.ylabel(r"$\epsilon(r)$")
plt.title("Quaternionic flux in Kerr spacetime (shooting)")
plt.legend(); plt.grid(ls='--', alpha=.3); plt.tight_layout(); plt.show()

# ---------- δc_g/c at 10 r_g ------------------------------------
f_LISA = 1e-2
k_hat  = 2*pi*f_LISA*r_g_m/c**2      # k 轉幾何單位
idx10  = np.argmin(np.abs(x_arr-10))
epsilon10 = ε_arr[idx10]
delta_c  = (epsilon10**2 * (a_hat/10)**2)/(2*k_hat**2)
print(f"\nε(10 r_g) ≈ {epsilon10:.2e}")
print(f"δc_g/c     ≈ {delta_c:.2e}")
